In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("HM1").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/10/19 00:53:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#!pip install pandas

In [3]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
# or: requests.get(url).content

resp = urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip")
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()

['household_power_consumption.txt']

In [4]:
import pandas as pd

pdata = pd.read_csv (zipfile.open('household_power_consumption.txt'), sep=';',
                   parse_dates={'data': ['Date', 'Time']},
                   infer_datetime_format=True,
                   na_values=['nan', '?'],
                   index_col='data')
pdata

/tmp/ipykernel_381/1105165950.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pdata = pd.read_csv (zipfile.open('household_power_consumption.txt'), sep=';',
/tmp/ipykernel_381/1105165950.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pdata = pd.read_csv (zipfile.open('household_power_consumption.txt'), sep=';',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
data,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0


In [5]:
#Spark dataframe

pd.DataFrame.iteritems = pd.DataFrame.items
data = spark.createDataFrame(pdata)
    
data.show()

print((data.count(), len(data.columns)))

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/types.py:238: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):
23/10/19 00:55:21 WARN TaskSetManager: Stage 0 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|              4.216|                0.418| 234.84|            18.4|           0.0|           1.0|          17.0|
|               5.36|                0.436| 233.63|            23.0|           0.0|           1.0|          16.0|
|              5.374|                0.498| 233.29|            23.0|           0.0|           2.0|          17.0|
|              5.388|                0.502| 233.74|            23.0|           0.0|           1.0|          17.0|
|              3.666|                0.528| 235.68|            15.8|           0.0|           1.0|          17.0|
|               3.52|                0.522| 235.02|            15.0|           0.0|     

23/10/19 00:55:24 WARN TaskSetManager: Stage 1 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


(2075259, 7)


In [6]:
# Find Null, None, NaN of All the DataFrame

from pyspark.sql.functions import col,isnan, when, count

data.select([when(isnan(c) | col(c).isNull(), c).alias(c) for c in data.columns]).show()

23/10/19 00:55:30 WARN TaskSetManager: Stage 3 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|               null|                 null|   null|            null|          null|          null|          null|
|               null|                 null|   null|            null|          null|          null|          null|
|               null|                 null|   null|            null|          null|          null|          null|
|               null|                 null|   null|            null|          null|          null|          null|
|               null|                 null|   null|            null|          null|          null|          null|
|               null|                 null|   null|            null|          null|     

In [7]:
# Find Count of Null, None, NaN of All DataFrame Columns
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

23/10/19 00:55:30 WARN TaskSetManager: Stage 4 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|              25979|                25979|  25979|           25979|         25979|         25979|         25979|
+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+



In [8]:
#Dropp rows with na value
print("Before ", (data.count(), len(data.columns)))
data = data.na.drop()
print("After ", (data.count(), len(data.columns)))

23/10/19 00:55:33 WARN TaskSetManager: Stage 6 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.
23/10/19 00:55:36 WARN TaskSetManager: Stage 8 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


Before  (2075259, 7)


After  (2049280, 7)


In [9]:
# Find Count of Null, None, NaN of All DataFrame Columns

from pyspark.sql.functions import col,isnan, when, count
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

23/10/19 00:55:39 WARN TaskSetManager: Stage 10 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|                  0|                    0|      0|               0|             0|             0|             0|
+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+



In [10]:
#Replacing missing value by the mean
"""from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = data.columns, outputCols = data.columns).setStrategy("mean").setMissingValue(0.0)
data = imputer.fit(data).transform(data)
data.show()
print((data.count(), len(data.columns)))"""

'from pyspark.ml.feature import Imputer\n\nimputer = Imputer(inputCols = data.columns, outputCols = data.columns).setStrategy("mean").setMissingValue(0.0)\ndata = imputer.fit(data).transform(data)\ndata.show()\nprint((data.count(), len(data.columns)))'

#### Output the minimum, maximum, and count of the following columns: ‘global active power’, ‘global reactive power’, ‘voltage’, and ‘global’ 

In [11]:
#Min, Max and Count of respective columns
data.agg({'Global_active_power': 'min',
          'Global_reactive_power': 'min',
          'Voltage': 'min',
          'Global_intensity': 'min'}).show()

data.agg({'Global_active_power': 'max',
          'Global_reactive_power': 'max',
          'Voltage': 'max',
          'Global_intensity': 'max'}).show()

data.agg({'Global_active_power': 'count',
          'Global_reactive_power': 'count',
          'Voltage': 'count',
          'Global_intensity': 'count'}).show()

23/10/19 00:55:42 WARN TaskSetManager: Stage 12 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+--------------------------+------------+------------------------+
|min(Global_intensity)|min(Global_reactive_power)|min(Voltage)|min(Global_active_power)|
+---------------------+--------------------------+------------+------------------------+
|                  0.2|                       0.0|       223.2|                   0.076|
+---------------------+--------------------------+------------+------------------------+



23/10/19 00:55:44 WARN TaskSetManager: Stage 14 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+--------------------------+------------+------------------------+
|max(Global_intensity)|max(Global_reactive_power)|max(Voltage)|max(Global_active_power)|
+---------------------+--------------------------+------------+------------------------+
|                 48.4|                      1.39|      254.15|                  11.122|
+---------------------+--------------------------+------------+------------------------+



23/10/19 00:55:48 WARN TaskSetManager: Stage 16 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+-----------------------+----------------------------+--------------+--------------------------+
|count(Global_intensity)|count(Global_reactive_power)|count(Voltage)|count(Global_active_power)|
+-----------------------+----------------------------+--------------+--------------------------+
|                2049280|                     2049280|       2049280|                   2049280|
+-----------------------+----------------------------+--------------+--------------------------+



#### Output the mean and standard deviation of these columns

In [12]:
#Mean and standard deviation of respective columns
data.agg({'Global_active_power': 'mean',
          'Global_reactive_power': 'mean',
          'Voltage': 'mean',
          'Global_intensity': 'mean'}).show()

data.agg({'Global_active_power': 'std',
          'Global_reactive_power': 'std',
          'Voltage': 'std',
          'Global_intensity': 'std'}).show()

23/10/19 00:55:51 WARN TaskSetManager: Stage 18 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+---------------------+--------------------------+------------------+------------------------+
|avg(Global_intensity)|avg(Global_reactive_power)|      avg(Voltage)|avg(Global_active_power)|
+---------------------+--------------------------+------------------+------------------------+
|    4.627759310588202|       0.12371447630388303|240.83985797451243|      1.0916150365005404|
+---------------------+--------------------------+------------------+------------------------+



23/10/19 00:55:54 WARN TaskSetManager: Stage 20 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+------------------------+-----------------------------+------------------+---------------------------+
|stddev(Global_intensity)|stddev(Global_reactive_power)|   stddev(Voltage)|stddev(Global_active_power)|
+------------------------+-----------------------------+------------------+---------------------------+
|       4.444396259786158|          0.11272197955071693|3.2399866790097174|          1.057294161093984|
+------------------------+-----------------------------+------------------+---------------------------+



#### Perform min-max normalization on the columns to generate normalized output

In [13]:
#MinMaxScaler with vector assembler and pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as func

columns_to_scale = ['Global_active_power', 'Global_reactive_power' ,'Voltage', 'Global_intensity']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(data)
scaledData = scalerModel.transform(data)

#Post-processing
names = {x + "_scaled": x for x in columns_to_scale}
scaledData = scaledData.select([func.col(c).alias(names[c]) for c in names.keys()])

scaledData.show()

23/10/19 00:55:58 WARN TaskSetManager: Stage 22 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.
23/10/19 00:56:03 WARN TaskSetManager: Stage 24 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.
23/10/19 00:56:06 WARN TaskSetManager: Stage 26 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.
23/10/19 00:56:09 WARN TaskSetManager: Stage 28 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.
23/10/19 00:56:13 WARN TaskSetManager: Stage 30 contains a task of very large size (33421 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------------+--------------------+--------------------+
| Global_active_power|Global_reactive_power|             Voltage|    Global_intensity|
+--------------------+---------------------+--------------------+--------------------+
|[0.37479630635524...| [0.30071942446043...| [0.376090468497577]|[0.37759336099585...|
|[0.47836320840123...| [0.3136690647482015]|[0.33699515347334...|[0.4730290456431536]|
|[0.4796306355241716]| [0.35827338129496...|[0.3260096930533117]|[0.4730290456431536]|
|[0.4808980626471121]| [0.3611510791366907]|[0.3405492730210021]|[0.4730290456431536]|
|[0.32500452652543...| [0.3798561151079137]|[0.40323101777059...|[0.32365145228215...|
| [0.311787072243346]| [0.3755395683453238]|[0.3819063004846531]|[0.30705394190871...|
|[0.3282636248415716]| [0.3741007194244605]|[0.3841680129240713]|[0.32365145228215...|
|[0.3280825638240087]| [0.3741007194244605]|[0.38836833602584...|[0.32365145228215...|
| [0.325185587543002]| [0.36690647482014...